# 🏁 SpectraMind V50 — 04 · Leaderboard & Submission

This notebook closes the loop for the **NeurIPS 2025 Ariel Data Challenge**:

- Aggregate **ablation/tuning** results and pick a best run
- Run **self-test** to validate the pipeline
- Generate a **submission** (μ, σ) with CLI
- Perform basic **schema checks** on the submission
- (Optional) **Upload** to Kaggle via API
- Append an entry to `v50_debug_log.md` for reproducibility

> CLI-first, Hydra-driven, DVC-aware. Cells auto-skip gracefully if CLI isn't on PATH.


## 🔧 Environment & Paths

In [ ]:
from pathlib import Path
import json, os, sys, platform, shutil
from datetime import datetime

ROOT = Path.cwd()
ART = ROOT / "artifacts"
ABLATE_DIR = ART / "ablation"
DIAG_DIR = ART / "diagnostics"
SUBMIT_DIR = ART / "submission"
SUBMIT_DIR.mkdir(parents=True, exist_ok=True)

LEADERBOARD_JSON = ABLATE_DIR / "leaderboard.json"
LEADERBOARD_CSV  = ABLATE_DIR / "leaderboard.csv"
BEST_RUN_JSON    = ABLATE_DIR / "best_run.json"   # optional
SUBMISSION_CSV   = SUBMIT_DIR / "submission.csv"
BUNDLE_ZIP       = SUBMIT_DIR / "submission_bundle.zip"
LOG_MD           = ROOT / "v50_debug_log.md"

env = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cwd": str(ROOT),
    "paths": {
        "ART": str(ART),
        "ABLATE_DIR": str(ABLATE_DIR),
        "DIAG_DIR": str(DIAG_DIR),
        "SUBMIT_DIR": str(SUBMIT_DIR),
        "LEADERBOARD_JSON": str(LEADERBOARD_JSON),
        "LEADERBOARD_CSV": str(LEADERBOARD_CSV),
        "BEST_RUN_JSON": str(BEST_RUN_JSON),
        "SUBMISSION_CSV": str(SUBMISSION_CSV),
        "BUNDLE_ZIP": str(BUNDLE_ZIP),
        "LOG_MD": str(LOG_MD),
    }
}
print(json.dumps(env, indent=2))


## 🩺 CLI sanity (optional)

In [ ]:
import shutil, subprocess

def check_cli(cmd="spectramind", args=["--version"]):
    exe = shutil.which(cmd)
    if not exe:
        print("⚠️ 'spectramind' CLI not found on PATH. Notebook will still run, but CLI calls will be skipped.")
        return {"available": False}
    try:
        out = subprocess.check_output([cmd] + args, stderr=subprocess.STDOUT, text=True, timeout=30)
        print(out)
        return {"available": True, "output": out}
    except Exception as e:
        print(f"⚠️ CLI call failed: {e}")
        return {"available": True, "error": str(e)}

cli_info = check_cli()
cli_info


## 📊 Aggregate ablation & pick best run

In [ ]:
import csv, math, json
from pathlib import Path

best = None
rows = []

# Try CSV first
if LEADERBOARD_CSV.exists():
    try:
        import pandas as pd
        df = pd.read_csv(LEADERBOARD_CSV)
        print("Loaded ablation leaderboard CSV:", LEADERBOARD_CSV, "shape=", df.shape)
        # Try common metric keys, prefer lower-is-better GLL if present
        metric_key = None
        for k in ["gll", "GLL", "score", "val_gll", "val_score", "mean_gll"]:
            if k in df.columns:
                metric_key = k
                break
        if metric_key:
            # lower GLL better; if score, assume higher is better and invert for sort flag
            ascending = True if "gll" in metric_key.lower() else False
            df_sorted = df.sort_values(metric_key, ascending=ascending)
            best = df_sorted.iloc[0].to_dict()
            display_cols = [c for c in df.columns if c in ["run_id", "short_id", "config", metric_key]]
            print("Top 5 by", metric_key)
            display(df_sorted[display_cols].head(5))
        else:
            print("No known metric column found; showing head:")
            display(df.head())
    except Exception as e:
        print("CSV parse failed:", e)

# Fallback to JSON
if best is None and LEADERBOARD_JSON.exists():
    try:
        with open(LEADERBOARD_JSON, "r", encoding="utf-8") as f:
            data = json.load(f)
        # Expect list of dicts; pick by gll or score
        metric_key = None
        if isinstance(data, list) and data:
            keys = list(data[0].keys())
            for k in ["gll", "GLL", "score", "val_gll", "val_score", "mean_gll"]:
                if k in keys:
                    metric_key = k
                    break
        if metric_key:
            def sort_key(d):
                v = d.get(metric_key, math.inf)
                try: return float(v)
                except: return math.inf
            # lower gll better; if score, higher better
            reverse = False if "gll" in metric_key.lower() else True
            data_sorted = sorted(data, key=sort_key, reverse=reverse)
            best = data_sorted[0]
            print("Top 3 candidates by", metric_key)
            for i, d in enumerate(data_sorted[:3]):
                rid = d.get("run_id") or d.get("short_id") or f"r{i}"
                print(i+1, rid, metric_key, d.get(metric_key))
        else:
            print("JSON leaderboard present but no known metric. Keys:", list(data[0].keys()) if isinstance(data, list) and data else None)
    except Exception as e:
        print("JSON parse failed:", e)

# Persist best run selection (optional)
if best:
    with open(BEST_RUN_JSON, "w", encoding="utf-8") as f:
        json.dump(best, f, indent=2)
    print("Best run saved to:", BEST_RUN_JSON)
else:
    print("⚠️ No ablation leaderboard found; proceeding without an explicit best-run selection.")


## ✅ Run pipeline self-test

In [ ]:
import subprocess, shutil

def run_selftest():
    exe = shutil.which("spectramind")
    if not exe:
        print("⚠️ spectramind CLI not found; skipping self-test.")
        return False
    # Try deep test first, fallback to fast
    cmds = [
        ["spectramind", "test", "--deep"],
        ["spectramind", "test"]
    ]
    for cmd in cmds:
        try:
            print("Running:", " ".join(cmd))
            subprocess.check_call(cmd, timeout=1800)
            print("✅ Self-test passed:", " ".join(cmd))
            return True
        except Exception as e:
            print("Self-test variant failed:", e)
    print("⚠️ All self-test variants failed.")
    return False

_ = run_selftest()


## 📦 Generate submission (μ/σ) via CLI

In [ ]:
import subprocess, shutil
from pathlib import Path

def generate_submission(out_csv: Path):
    exe = shutil.which("spectramind")
    if not exe:
        print("⚠️ spectramind CLI not found; skipping submission generation.")
        return False
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    cmd = ["spectramind", "submit", "--out", str(out_csv), "--include-diagnostics", "true"]
    # If BEST_RUN_JSON exists and includes a config or short_id, pass it through as override if supported
    if BEST_RUN_JSON.exists():
        try:
            import json
            best = json.loads(BEST_RUN_JSON.read_text())
            # attempt a generic override key if available
            if "config" in best and isinstance(best["config"], str):
                cmd += ["--config", best["config"]]
            elif "short_id" in best:
                cmd += ["--run-id", str(best["short_id"])]
        except Exception as e:
            print("Note: couldn't parse BEST_RUN_JSON for overrides:", e)
    print("Running:", " ".join(cmd))
    try:
        subprocess.check_call(cmd, timeout=6*3600)  # allow long run
        print("✅ Submission written:", out_csv)
        return True
    except Exception as e:
        print("❌ Submission generation failed:", e)
        return False

ok = generate_submission(SUBMISSION_CSV)
ok


## 🔎 Basic submission schema check

In [ ]:
import pandas as pd
import numpy as np

def validate_submission(path: Path):
    if not path.exists():
        print("❌ No submission file found at", path)
        return False
    try:
        df = pd.read_csv(path)
    except Exception as e:
        print("❌ CSV read failed:", e)
        return False

    print("Submission shape:", df.shape)
    # Very generic checks: non-empty, numeric columns present
    if df.empty:
        print("❌ Submission is empty.")
        return False

    # Try to detect numeric prediction columns (μ/σ); require at least a few numeric columns
    num_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
    if len(num_cols) < 10:
        print("⚠️ Fewer than 10 numeric columns detected; ensure μ/σ columns are included.")
    else:
        # Check no NaNs in prediction columns
        nan_counts = df[num_cols].isna().sum().sum()
        if nan_counts > 0:
            print(f"⚠️ Found {nan_counts} NaNs in numeric columns; please investigate.")

    # Optional: enforce monotonic ID or required column name heuristics if known
    print("✅ Basic checks completed.")
    return True

_ = validate_submission(SUBMISSION_CSV)


## 🗜️ Bundle submission + diagnostics

In [ ]:
import zipfile

def bundle_zip(bundle: Path, submission: Path, extras: list[Path] = None):
    extras = extras or []
    with zipfile.ZipFile(bundle, "w", compression=zipfile.ZIP_DEFLATED) as z:
        if submission.exists():
            z.write(submission, submission.name)
        else:
            print("⚠️ No submission to include.")
        for p in extras:
            if p.exists():
                if p.is_file():
                    z.write(p, p.name)
                else:
                    # add directory contents flatly
                    for child in p.rglob("*"):
                        if child.is_file():
                            arc = child.relative_to(p.parent)
                            z.write(child, str(arc))
    print("Bundle written:", bundle)

extras = [DIAG_DIR] if DIAG_DIR.exists() else []
bundle_zip(BUNDLE_ZIP, SUBMISSION_CSV, extras)


## ⤴️ (Optional) Push to Kaggle via API

In [ ]:
# This cell attempts a Kaggle upload if configured.
# Prerequisites:
#  - Install kaggle:  pip install kaggle
#  - Place Kaggle credentials (~/.kaggle/kaggle.json) or set KAGGLE_USERNAME / KAGGLE_KEY env vars
#  - Set COMPETITION to the exact competition slug

import os, subprocess, shutil

COMPETITION = os.environ.get("KAGGLE_COMPETITION", "ariel-data-challenge-2025")

def kaggle_submit(csv_path: Path, message: str = "SpectraMind V50 submission"):
    if not csv_path.exists():
        print("⚠️ No CSV to submit.")
        return False
    # Try kaggle CLI
    exe = shutil.which("kaggle")
    if not exe:
        print("ℹ️ kaggle CLI not found; skipping Kaggle upload.")
        return False
    try:
        cmd = ["kaggle", "competitions", "submit", "-c", COMPETITION, "-f", str(csv_path), "-m", message]
        print("Running:", " ".join(cmd))
        subprocess.check_call(cmd, timeout=600)
        print("✅ Kaggle submission attempted.")
        return True
    except Exception as e:
        print("⚠️ Kaggle submission failed:", e)
        return False

# Disabled by default; uncomment to attempt submission
# _ = kaggle_submit(SUBMISSION_CSV, "SpectraMind V50 auto-generated submission")


## 🧾 Append run metadata to `v50_debug_log.md`

In [ ]:
from datetime import datetime

entry = f"""### Notebook: 04_leaderboard_and_submission.ipynb
- timestamp: {datetime.now().isoformat(timespec="seconds")}
- cwd: {ROOT}
- python: {platform.python_version()}
- actions:
  - ablation_leaderboard_csv: {LEADERBOARD_CSV.exists()}
  - ablation_leaderboard_json: {LEADERBOARD_JSON.exists()}
  - best_run_json: {BEST_RUN_JSON.exists()}
  - submission_csv_exists: {SUBMISSION_CSV.exists()}
  - bundle_zip_exists: {BUNDLE_ZIP.exists()}
"""

try:
    with open(LOG_MD, "a", encoding="utf-8") as f:
        f.write(entry + "\n")
    print(f"Appended notebook log entry to {LOG_MD}")
except Exception as e:
    print(f"⚠️ Could not append to {LOG_MD}: {e}")
